# Figure 4: Distribution of extinction times

Generate **figures/hist.pdf** and calculate summary statistics for meltdown
under default mutational parameters.

In [ ]:
from doomed import *

### Expected extinction time

In [ ]:
# Default parameters
u = 0.023
s = 0.075
Z0 = List([1000])

In [ ]:
m, v = extinction_time(Z0, 15, s, u, 1e-8)
print('Expected value:', round(m, 3))
print('            CV:', round(np.sqrt(v) / m, 2))

### Expected time of first click

In [ ]:
m2, v2 = click_time(1000, 0, s, u, 1e-8)
print('Expected value:', round(m2, 3), '   ', round(m2/m * 100, 1), '%')
print('            CV:', round(np.sqrt(v2) / m2, 2))

### Expected population size at extinction time

In [ ]:
ENt(Z0, 193, s, u)

### Time required for expected population size to drop below 1

Checking approach proposed by Lansch-Justen et al. (2022).

Estimate: 311 generations.

In [ ]:
for n in range(309, 314):
    print(n, ENt(Z0, n, s, u))

### Probability that a population is extinct at that time 

In [ ]:
prob_extinct(Z0, 311, 50, s, u)

### Probability density

In [ ]:
red = sns.xkcd_rgb["pale red"]
theoryT = []
theoryP = []
binw = 5
t1 = 0
p1 = 0
for i in range(1, 121):
    t2 = i * binw
    p2 = prob_extinct(Z0, t2, 15, s, u)
    theoryT.append((t1+t2)/2)
    theoryP.append((p2-p1) / binw)
    t1 = t2
    p1 = p2
plt.plot(theoryT, theoryP, color=red)
plt.ylabel('Probability density')
plt.xlabel('Extinction time (generations)');

### Stochastic simulations

You can skip directly to the "Load data" section below.

In [ ]:
np.random.seed(2102023)
tt = []
for i in range(10000):
    tmpN, tmpZ, t = to_extinction(Z0, s, u)
    tt.append(t)
t = np.array(tt)
t.mean(), t.var(ddof=1), len(t)

### Save data

Save extinction times.

In [ ]:
with open('../data/hist.npy', 'wb') as f:
    np.save(f, t)

### Load data

Extinction times can be retrieved directly.

In [ ]:
t = np.load('../data/hist.npy')
t.mean(), t.var(ddof=1), len(t)

In [ ]:
t.min(), t.max()

In [ ]:
bins = np.arange(10 * np.floor((t.min()-1)/10), 10 * np.ceil((t.max()+1)/10) + 1, 10)
bins

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax.set_position([.212, .175, .78, .817])
ax.hist(t, bins=bins, density=True, color='0.7')
ax.plot(theoryT, theoryP, color=red)
ax.set_ylabel('Probability')
ax.set_xlabel('Extinction time (generations)')
fig.savefig('../figures/hist.pdf');